In [1]:
# primary settings
state = 'AL'
district_type = 'CD'
level = 'block'
minority = 'Black'

In [2]:
filepath = 'C:\\districting-data-2020-reprojection\\'
filename = state + '_' + level + '.json'

from read import read_graph_from_json
G = read_graph_from_json( filepath + filename )
print("number of nodes, edges:",G.number_of_nodes(),G.number_of_edges())

number of nodes, edges: 185976 461910


In [3]:
# read Plan D from file
filename = "PlanDDec26.csv"
geoid_to_node = { G.nodes[i]['GEOID20'] : i for i in G.nodes }

from number_of_districts import number_of_districts
k = number_of_districts[state, district_type]
districts = [ list() for j in range(k) ]

import pandas
csvFile = pandas.read_csv( filename, skipinitialspace = True )
unassigned = list()
for index, row in csvFile.iterrows():

    # what is the block's geoid?
    g = str( row['GEOID20'] )
    if len(g) < 15: # fix issue with leading zeros
        g = '0' + g

    # which node does this correspond to?
    i = geoid_to_node[g]

    # which district is it in?
    j = str( row['District'] ) 

    # is it unassigned??
    if j in { 'ZZ', 'ZZZ' }:
        unassigned.append(i)
        continue
    else:
        districts[int(j)-1].append(i)

assert len(unassigned)==0

In [4]:
# read more-detailed version of block-level graph
#   to get additional data fields (VAP and MVAP)
GB = read_graph_from_json( "al_blocks.json", update_population=False, rescale_distance=False )

for i in G.nodes:
    #assert G.nodes[i]['GEOID20'] == GB.nodes[i]['GEOID20']
    G.nodes[i]['VAP'] = GB.nodes[i]['VAP20']
    G.nodes[i]['MVAP'] = GB.nodes[i]['APBVAP20']

del GB # free up some memory
    
from metrics import report_metrics
report_metrics(G, districts, minority=minority, verbose=True)


District Black percentages:
0 15.36 
1 50.05 ***
2 23.96 
3 8.58 
4 16.02 
5 15.37 
6 51.73 ***
-> 2 majority-Black districts

Counties split (by fips code): ['01117', '01097', '01083', '01073', '01113']
County splits (by fips code): {'01117': 1, '01097': 1, '01083': 1, '01073': 2, '01113': 1}
-> 5 counties split a total of 6 times

District Polsby-Popper scores:
0 0.1325
1 0.1546
2 0.2683
3 0.3649
4 0.381
5 0.1934
6 0.2691
-> average Polsby-Popper score of 0.252


In [5]:
populations = [ sum( G.nodes[i]['TOTPOP'] for i in district ) for district in districts ]
G._L = min(populations)
G._U = max(populations)
G._k = k
G._state = state
G._level = level
print("Using k, L, U =",G._k, G._L, G._U)

Using k, L, U = 7 717754 717755


In [6]:
# Cleanup the seed PlanD, making it more compact, 
#   with 1-hop and 2-hop local search
from cleanup import mip_local_search
from export import export_to_csv

In [7]:
districts = mip_local_search(G, districts, h=1, minority=minority)
report_metrics(G, districts, minority, verbose=True)

labeling = { i : j for j in range(len(districts)) for i in districts[j] }
csv_filename = "PlanD1.csv"
export_to_csv(G, labeling, csv_filename=csv_filename)

Applying MIP-based local search to improve the MIP warm start (hop parameter: 1)...
iter 		 obj 		 time
Set parameter Username
Academic license - for non-commercial use only - expires 2025-04-16
1 	 4.27655621 	 600.47
2 	 4.20884918 	 600.53
3 	 4.19745767 	 600.58
4 	 4.18722513 	 600.49
5 	 4.18376660 	 600.50
6 	 4.17792914 	 600.45
7 	 4.17570322 	 600.54
8 	 4.17570322 	 600.75

District Black percentages:
0 15.4 
1 50.0 ***
2 23.97 
3 8.59 
4 16.01 
5 15.39 
6 51.69 ***
-> 2 majority-Black districts

Counties split (by fips code): ['01117', '01097', '01083', '01073', '01113']
County splits (by fips code): {'01117': 1, '01097': 1, '01083': 1, '01073': 2, '01113': 1}
-> 5 counties split a total of 6 times

District Polsby-Popper scores:
0 0.1416
1 0.1647
2 0.2779
3 0.4345
4 0.4437
5 0.2149
6 0.3039
-> average Polsby-Popper score of 0.283
Exporting to csv...


In [8]:
districts = mip_local_search(G, districts, h=2, minority=minority)
report_metrics(G, districts, minority, verbose=True)

labeling = { i : j for j in range(len(districts)) for i in districts[j] }
csv_filename = "PlanD2.csv"
export_to_csv(G, labeling, csv_filename=csv_filename)

Applying MIP-based local search to improve the MIP warm start (hop parameter: 2)...
iter 		 obj 		 time
1 	 4.08690934 	 600.76
2 	 4.04422097 	 601.88
3 	 4.01193945 	 600.80
4 	 4.00908415 	 601.86
5 	 4.00255212 	 600.72
6 	 3.99897721 	 600.77
7 	 3.99685611 	 600.75
8 	 3.98777366 	 600.68
9 	 3.98541707 	 600.72
10 	 3.98541707 	 600.67

District Black percentages:
0 15.43 
1 50.01 ***
2 23.91 
3 8.57 
4 16.01 
5 15.52 
6 51.57 ***
-> 2 majority-Black districts

Counties split (by fips code): ['01117', '01097', '01083', '01073', '01113']
County splits (by fips code): {'01117': 1, '01097': 1, '01083': 1, '01073': 2, '01113': 1}
-> 5 counties split a total of 6 times

District Polsby-Popper scores:
0 0.1476
1 0.1732
2 0.2844
3 0.4789
4 0.4827
5 0.224
6 0.3115
-> average Polsby-Popper score of 0.3003
Exporting to csv...


In [9]:
districts = mip_local_search(G, districts, h=3, minority=minority, time_limit=3600)
report_metrics(G, districts, minority, verbose=True)

labeling = { i : j for j in range(len(districts)) for i in districts[j] }
csv_filename = "PlanD3.csv"
export_to_csv(G, labeling, csv_filename=csv_filename)

Applying MIP-based local search to improve the MIP warm start (hop parameter: 3)...
iter 		 obj 		 time
1 	 3.86604932 	 3602.91
2 	 3.84412422 	 3602.22
3 	 3.82248305 	 3603.13
4 	 3.77843980 	 3603.35
5 	 3.77770504 	 3603.49
6 	 3.76351197 	 3602.47
7 	 3.73962458 	 3602.25
8 	 3.73696769 	 2166.99
9 	 3.73632913 	 3602.86
10 	 3.73632913 	 3601.08

District Black percentages:
0 15.44 
1 50.0 ***
2 23.95 
3 8.34 
4 16.01 
5 15.77 
6 51.54 ***
-> 2 majority-Black districts

Counties split (by fips code): ['01117', '01097', '01083', '01073', '01113']
County splits (by fips code): {'01117': 1, '01097': 1, '01083': 1, '01073': 2, '01113': 1}
-> 5 counties split a total of 6 times

District Polsby-Popper scores:
0 0.1566
1 0.1688
2 0.319
3 0.5455
4 0.4827
5 0.2653
6 0.3298
-> average Polsby-Popper score of 0.3239
Exporting to csv...


In [10]:
districts = mip_local_search(G, districts, h=4, minority=minority, time_limit=3600)
report_metrics(G, districts, minority, verbose=True)

labeling = { i : j for j in range(len(districts)) for i in districts[j] }
csv_filename = "PlanD4.csv"
export_to_csv(G, labeling, csv_filename=csv_filename)

Applying MIP-based local search to improve the MIP warm start (hop parameter: 4)...
iter 		 obj 		 time
1 	 3.71233952 	 3601.18
2 	 3.69160737 	 3601.11
3 	 3.67334938 	 3603.13
4 	 3.65954206 	 3601.39
5 	 3.64943832 	 3601.98
6 	 3.64943832 	 3603.06

District Black percentages:
0 15.43 
1 50.0 ***
2 23.96 
3 8.21 
4 16.14 
5 15.77 
6 51.54 ***
-> 2 majority-Black districts

Counties split (by fips code): ['01117', '01097', '01083', '01073', '01113']
County splits (by fips code): {'01117': 1, '01097': 1, '01083': 1, '01073': 2, '01113': 1}
-> 5 counties split a total of 6 times

District Polsby-Popper scores:
0 0.165
1 0.1735
2 0.3211
3 0.5505
4 0.4941
5 0.267
6 0.3306
-> average Polsby-Popper score of 0.3288
Exporting to csv...
